We can import the gui and instantiate a new session. If there is a saved session with the same name, it will be automatically loaded. To see the gui, use the `.draw()` method.

In [1]:
%matplotlib inline
from ironflow import GUI

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
gui = GUI('example')

In [3]:
gui.draw()

In [4]:
gui.selected_node.outputs.values.project.remove_jobs(silently=True, recursive=True)

  0%|          | 0/9 [00:00<?, ?it/s]

We can also extend ironflow with new nodes on-the-fly. Most of the tools you should need are stored under `ironflow.custom_nodes`. Once we register a new node from a notebook, it immediately shows up under the `__main__` tab in the node selector.

In [5]:
from ironflow.node_tools import Node, NodeInputBP, NodeOutputBP, dtypes, input_widgets

class My_Node(Node):
    title = "MyUserNode"
    init_inputs = [
        NodeInputBP(dtype=dtypes.Integer(default=1), label="foo")
    ]
    init_outputs = [
       NodeOutputBP(label="bar")
    ]
    color = 'cyan'

    def update_event(self, inp=-1):
        self.set_output_val(0, self.input(0) + 42)

gui.register_node(My_Node)

If we save a session with a custom node, the same node needs to registered again *before* we load that session! To instantiate and load such a saved session all at once, extra node packages can be included using the optional `extra_node_packages` argument. This takes a `list` of node packages, which should either be a list of nodes that are children of `Node` (as in the example below) -- these appear under `__main__` in the gui, or a python module or path to a .py file. When registering nodes from a module or file, only those that inherit from `Node` *and* have a class name ending in `_Node` will be registered (this allows you to have your own node class templates and avoid loading the template itself by simply using regular python CamelCase naming conventions and avoiding ending in `_Node`). 

In [6]:
gui2 = GUI("example_with_custom_node", extra_nodes_packages=[[My_Node]])

In [7]:
gui2.draw()

Note: When registering nodes from a module or file, they appear in the tab based on the end of the module/file path (excluding the .py convention). This is intentional since nodes from multiple sources may be conceptually linked, so they get grouped with every other node that has the same location terminus, but it's possible it could lead to naming conflicts. You're already able to override this with the underlying `GUI.register_nodes` method, which allows you to specify your own location using the optional `node_group` argument. In a future update we plan to provide the same capability when registering nodes at initialization.